## Penggunaan Decision Tree Algorithm

### Bagaimana menerapkan Decision Tree dengan Scikit-Learn

+ Perlu di ingat bahwa algorithm ini dapat menghasilkan model prediksi yang tidak akurat apbila kelas kelas dalam trining dataset tidak seimbang jumlahnya.
+ Jika demikian halnya dibutuhkan proses 'balancing' melalui tahapan membuang/ menambahkan sebagian data dari salah satu kelas.

Uji coba kali ini akan menggunakan dataset 'decisiontree_ch6.csv' yang dapat di download pada halaman ini.

In [1]:
%matplotlib inline

# data manuipulation
import numpy as np
import pandas as pd

# modeling utilities
import pydotplus 
from sklearn import tree
from sklearn import metrics
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


# plotting libraries
import seaborn as sn
import matplotlib.pyplot as plt
import seaborn as sn


sn.set_style('whitegrid')
sn.set_context('talk')
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (30, 10),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}

plt.rcParams.update(params)

# Load dataset

In [2]:
df1 = pd.read_csv('decisiontree_ch6.csv')
print ('Shape of dataset::{}'.format(df1.shape))

Shape of dataset::(40, 5)


## Preprocessing
+ Standarize column names
+ Encode Categoricals using One Hot Encoding

### Standarize column names

In [3]:
df1.rename(columns={'ID':'Car_id',
                   'mesin':'Car_mesin',
                   'bangku':'Car_shit',
                   'penggerak':'Force_power',
                   'label':'Label'}, inplace=True)

In [4]:
df1.head()

,Car_id,Car_mesin,Car_shit,Force_power,Label
0,1,bensin,4,depan,sedan
1,2,bensin,2,depan,sedan
2,3,bensin,8,belakang,minibus
3,4,diesel,6,depan,minibus
4,5,bensin,5,belakang,minibus


### Note :
 Karena isi feature masih ada yang berupa nilai alfanumerik:
 + bensin
 + depan
 + sedan
 
 Maka kita perlu lakukan transformasi berupa encoding ke dalam angka "0" dan "1"

In [5]:
encoding = {"Car_mesin":{"bensin":0, "diesel":1},
           "Force_power":{"depan":0, "belakang":1}}
df1.replace(encoding,  inplace=True)

In [6]:
df1.head()

,Car_id,Car_mesin,Car_shit,Force_power,Label
0,1,0,4,0,sedan
1,2,0,2,0,sedan
2,3,0,8,1,minibus
3,4,1,6,0,minibus
4,5,0,5,1,minibus


### Pemisahan fiture 
+ Ketiga fiture ("Car_mesin","Car_shit","Force_power") akan kita simpan dalam variable (X)
+ Label akan disimpan dalam variable (y) 

In [7]:
X = df1.drop(['Car_id','Label'],axis=1)
y = df1['Label']

In [8]:
import sklearn.model_selection as ms
X_train,X_test,y_train,y_test = ms.train_test_split(X,y, test_size=0.2)

### Pengukuran test Split awal
+ Persiapan implementasi Decision Tree dengan "DecisionTreeClassifier"
+ Ujicoba awal melatih model dengan menggunakan Entropy dalam proses spliting
+ Kedalaman dibatasi maks 5

In [9]:
import sklearn.tree as tree
model1 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)
model1.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=5)

### Test dataset
+ Model yang sudah terbagun akan di test dengan (X_test), hasil prediksi akan di simpan pada y_prediksi

In [10]:
y_prediksi = model1.predict(X_test)
y_prediksi

array(['minibus', 'minibus', 'sedan', 'sedan', 'minibus', 'sedan',
       'sedan', 'sedan'], dtype=object)

### Note : 
+ Hasil ini akan di bandingkan dengan label (y)    

In [11]:
import sklearn.metrics as met
met.accuracy_score(y_test,y_prediksi)

1.0

* Hasil ini sudah sangat akurat dnegan ditandai angka akurasi model bernilai '1' dengan demikian untuk kasus ini pemilihan ktereria spliting tersebut sudah tepat. 
* Adapun jika kurang teopat bisa kita bandingkan dnegan pemilihan metoda 'gini index' untuk di bandingkan angka akurasi modelnya

## Visualisasi

In [12]:
import pydotplus as pp
labels = ['Car_mesin','Car_shit','Force_power']
dot_data = tree.export_graphviz(model1, out_file=None, feature_names=labels, filled=True, rounded=True)
graph = pp.graph_from_dot_data(dot_data)
graph.write_png('decisiontree.png')

True

from IPython.display import Image
![title](decisiontree.png)

### Penjelasan

+ Node root yang dipilih oleh algorithm adalah 'Jumlah bangku' < 5 menghasilkan kelompok True di sebelah kiri 15 sample nilai entropy = 0.722
+ Ditingkat bawahnya proses spliting diulang,dan feature yang di ambil 'Tenaga penggerak'<1 atau (depan) menghasilkan kelompok True di sebelah kiri 14 sample dengan nilai entropy = 0.592
+ Demikian seterusnya hingga di peroleh leaf node tercapai